# Atividade:Analisador Morfológico








1.Pré-processamento de dados: rotular usando predefinições as palavras mais óbvias
 *   1.1 Definição de funções de pré-processamento
 *   1.2 Extrair,tratar e préprocessar conjunto de dados
 *   1.3 Calcular a acurácia do pré-processamento e decidir se é viável utilizá-lo

2.Machine Learning:
 *   2.1 Seleção de features (entrada) e target (saída)
 *  2.2 Seleção do modelo: utilizei árvores de decisão, pois apresentam baixo custo computacional e grande eficiência para este caso, uma vez que se baseiam em decisões, o que possui semelhanças com a forma como a língua portuguesa lida com a gramática.Utilizei grid search para o pos-tagging para descobrir os melhores parâmetros,mas,após achar estes parâmetros retirei o gridsearch devido o alto custo computacional para uma operação que pode ser feita uma única vez.
 *  2.3 Treinamento
 *  2.4 Acurácia do modelo

*Para o algoritmo de flexões, foi necessário dividir o target (dicionário com vários atributos,logo se trata de um caso de classificação multipla) em colunas, aplicar one-hot encoding(valores categóricos que não seguem ordem,logo uma matriz espaçada aumentaria a eficiência do algoritmo) e realizar o treinamento para cada coluna.

3.Criação da função Analisador Morfológico, utilizando tanto o pré-processamento quanto os modelos de machine learning



#1.Pré-processamento

In [ ]:
!pip install conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-train.conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-test.conllu

--2024-08-18 17:51:04--  http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-train.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.149.154
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.149.154|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7444078 (7.1M)
Saving to: ‘pt_porttinari-ud-train.conllu’

pt_porttinari-ud-tr 100%[===================>]   7.10M   531KB/s    in 34s     

2024-08-18 17:51:39 (211 KB/s) - ‘pt_porttinari-ud-train.conllu’ saved [7444078/7444078]

--2024-08-18 17:51:39--  http://marlovss.work.gd:8080/tomorrow/aula2/pt_porttinari-ud-test.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.149.154
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.149.154|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2120477 (2.0M)
Saving to: ‘pt_porttinari-ud-test.conllu’

pt_porttinari-ud-te 100%[===================>]   2.02M   523KB/s    in 4.5s    

2024-08-18 17:51:44 (4

In [ ]:
import conllu
import itertools as it
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np


1.1 Definindo funções

In [ ]:
class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

class CoNLLU:
    def __init__(self, files):
        self.words = []
        self.sentences = []
        for f in files:
            with open(f, encoding='utf-8') as file:
                parsed = conllu.parse(file.read())
            sents = [[AttributeDict(form=token['form'], lemma=token['lemma'], pos=token['upos'], feats=token['feats']) for token in tokenlist if token['upos'] != '_'] for tokenlist in parsed]
            self.sentences.extend(sents)
            self.words.extend([word for sent in sents for word in sent])
        self.pos_tags = set([word.pos for word in self.words])
        self.feats_dict = {pos: set(it.chain.from_iterable([list(word.feats.keys()) for word in self.words if word.pos == pos and word.feats is not None])) for pos in self.pos_tags}
def extract_data(conllu_data):
    texts = []
    pos_tags = []
    lemmas = []
    feats = []
    for word in conllu_data.words:
        texts.append(word.form)
        pos_tags.append(word.pos)
        lemmas.append(word.lemma)
        feats.append(word.feats if word.feats is not None else {})
    return texts, pos_tags, lemmas, feats


In [ ]:

def preprocess_token(token):
    if re.match(r'^[\.,;:!?\'\"\(\)\[\]\{\}»«]', token):
        return ('PUNCT', token, token, {})
    if re.match(r'^[\@\#\$\%\&\*\+\=\-\_\^\~\|\\\/]+$', token):
        return ('SYM', token, token, {})

    pronouns = {
        'eu': ('PRON', 'eu', 'eu', {'Gender': 'Masc', 'Number': 'Sing', 'PronType': 'Prs'}),
        'tu': ('PRON', 'tu', 'tu', {'Gender': 'Masc', 'Number': 'Sing', 'PronType': 'Prs'}),
        'ele': ('PRON', 'ele', 'ele', {'Gender': 'Masc', 'Number': 'Sing', 'PronType': 'Prs'}),
        'ela': ('PRON', 'ela', 'ela', {'Gender': 'Fem', 'Number': 'Sing', 'PronType': 'Prs'}),
        'nós': ('PRON', 'nós', 'nós', {'Gender': 'Masc', 'Number': 'Plur', 'PronType': 'Prs'}),
        'vós': ('PRON', 'vós', 'vós', {'Gender': 'Masc', 'Number': 'Plur', 'PronType': 'Prs'}),
        'eles': ('PRON', 'eles', 'eles', {'Gender': 'Masc', 'Number': 'Plur', 'PronType': 'Prs'}),
        'elas': ('PRON', 'elas', 'elas', {'Gender': 'Fem', 'Number': 'Plur', 'PronType': 'Prs'}),
        'meu': ('PRON', 'meu', 'meu', {'Gender': 'Masc', 'Number': 'Sing', 'PronType': 'Prs'}),
        'minha': ('PRON', 'minha', 'minha', {'Gender': 'Fem', 'Number': 'Sing', 'PronType': 'Prs'}),
        'teu': ('PRON', 'teu', 'teu', {'Gender': 'Masc', 'Number': 'Sing', 'PronType': 'Prs'}),
        'tua': ('PRON', 'tua', 'tua', {'Gender': 'Fem', 'Number': 'Sing', 'PronType': 'Prs'}),
        'seu': ('PRON', 'seu', 'seu', {'Gender': 'Masc', 'Number': 'Sing', 'PronType': 'Prs'}),
        'sua': ('PRON', 'sua', 'sua', {'Gender': 'Fem', 'Number': 'Sing', 'PronType': 'Prs'}),
        'nosso': ('PRON', 'nosso', 'nosso', {'Gender': 'Masc', 'Number': 'Plur', 'PronType': 'Prs'}),
        'nossa': ('PRON', 'nossa', 'nossa', {'Gender': 'Fem', 'Number': 'Plur', 'PronType': 'Prs'}),
        'vosso': ('PRON', 'vosso', 'vosso', {'Gender': 'Masc', 'Number': 'Plur', 'PronType': 'Prs'}),
        'vossa': ('PRON', 'vossa', 'vossa', {'Gender': 'Fem', 'Number': 'Plur', 'PronType': 'Prs'})
    }
    if token.lower() in pronouns:
        return pronouns[token.lower()]

    if token.lower().endswith('mente'):
        return ('ADV', token, token, {})

    prepositions = {'a', 'ante', 'após', 'até', 'com', 'contra', 'de', 'desde', 'em', 'entre', 'para', 'per', 'perante', 'por', 'sem', 'sob', 'sobre', 'trás'}
    if token.lower() in prepositions:
        return ('ADP', token, token, {})

    determiners = {
        'o': ('DET', 'o', 'o', {'Gender': 'Masc', 'Number': 'Sing'}),
        'a': ('DET', 'a', 'a', {'Gender': 'Fem', 'Number': 'Sing'}),
        'os': ('DET', 'os', 'os', {'Gender': 'Masc', 'Number': 'Plur'}),
        'as': ('DET', 'as', 'as', {'Gender': 'Fem', 'Number': 'Plur'}),
        'um': ('DET', 'um', 'um', {'Gender': 'Masc', 'Number': 'Sing'}),
        'uma': ('DET', 'uma', 'uma', {'Gender': 'Fem', 'Number': 'Sing'}),
        'uns': ('DET', 'uns', 'uns', {'Gender': 'Masc', 'Number': 'Plur'}),
        'umas': ('DET', 'umas', 'umas', {'Gender': 'Fem', 'Number': 'Plur'}),
        'meu': ('DET', 'meu', 'meu', {'Gender': 'Masc', 'Number': 'Sing'}),
        'minha': ('DET', 'minha', 'minha', {'Gender': 'Fem', 'Number': 'Sing'}),
        'teu': ('DET', 'teu', 'teu', {'Gender': 'Masc', 'Number': 'Sing'}),
        'tua': ('DET', 'tua', 'tua', {'Gender': 'Fem', 'Number': 'Sing'}),
        'seu': ('DET', 'seu', 'seu', {'Gender': 'Masc', 'Number': 'Sing'}),
        'sua': ('DET', 'sua', 'sua', {'Gender': 'Fem', 'Number': 'Sing'}),
        'nosso': ('DET', 'nosso', 'nosso', {'Gender': 'Masc', 'Number': 'Sing'}),
        'nossa': ('DET', 'nossa', 'nossa', {'Gender': 'Fem', 'Number': 'Sing'}),
        'vosso': ('DET', 'vosso', 'vosso', {'Gender': 'Masc', 'Number': 'Sing'}),
        'vossa': ('DET', 'vossa', 'vossa', {'Gender': 'Fem', 'Number': 'Sing'}),
        'este': ('DET', 'este', 'este', {'Gender': 'Masc', 'Number': 'Sing'}),
        'esta': ('DET', 'esta', 'esta', {'Gender': 'Fem', 'Number': 'Sing'}),
        'esse': ('DET', 'esse', 'esse', {'Gender': 'Masc', 'Number': 'Sing'}),
        'essa': ('DET', 'essa', 'essa', {'Gender': 'Fem', 'Number': 'Sing'}),
        'aquele': ('DET', 'aquele', 'aquele', {'Gender': 'Masc', 'Number': 'Sing'}),
        'aquela': ('DET', 'aquela', 'aquela', {'Gender': 'Fem', 'Number': 'Sing'}),
        'isso': ('DET', 'isso', 'isso', {'Gender': 'Neut', 'Number': 'Sing'}),
        'isto': ('DET', 'isto', 'isto', {'Gender': 'Neut', 'Number': 'Sing'}),
        'aquilo': ('DET', 'aquilo', 'aquilo', {'Gender': 'Neut', 'Number': 'Sing'})
    }
    if token.lower() in determiners:
        return determiners[token.lower()]

    cconj = {'e', 'ou', 'mas', 'porém', 'todavia', 'contudo', 'entretanto', 'nem', 'logo', 'portanto'}
    if token.lower() in cconj:
        return ('CCONJ', token, token, {})

    sconj = {'que', 'quando', 'como', 'se', 'porque', 'embora', 'conquanto', 'desde', 'enquanto'}
    if token.lower() in sconj:
        return ('SCONJ', token, token, {})

    if re.match(r'^\d+$', token) or token.lower() in {'um', 'dois', 'três', 'quatro', 'cinco', 'seis', 'sete', 'oito', 'nove', 'dez', 'primeiro', 'segundo', 'terceiro', 'quarto', 'quinto', 'sexto', 'sétimo', 'oitavo', 'nono', 'décimo', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'}:
        return ('NUM', token, token, {})

    if token.istitle():
        return ('PROPN', token, token, {})

    if token.lower().endswith(('dade', 'idade', 'ção', 'mento', 'menta', 'ência', 'ância', 'ez', 'eza', 'ismo', 'ista', 'ção', 'são', 'oso', 'osa')):
        return ('NOUN', token, token, {})

    verb_flexions = {
        'sou': ('VERB', 'ser', 'sou', {'Person': '1', 'Number': 'Sing', 'Tense': 'Pres', 'Mood': 'Ind'}),
        'és': ('VERB', 'ser', 'és', {'Person': '2', 'Number': 'Sing', 'Tense': 'Pres', 'Mood': 'Ind'}),
        'é': ('VERB', 'ser', 'é', {'Person': '3', 'Number': 'Sing', 'Tense': 'Pres', 'Mood': 'Ind'}),
        'somos': ('VERB', 'ser', 'somos', {'Person': '1', 'Number': 'Plur', 'Tense': 'Pres', 'Mood': 'Ind'}),
        'sois': ('VERB', 'ser', 'sois', {'Person': '2', 'Number': 'Plur', 'Tense': 'Pres', 'Mood': 'Ind'}),
        'são': ('VERB', 'ser', 'são', {'Person': '3', 'Number': 'Plur', 'Tense': 'Pres', 'Mood': 'Ind'}),
        'fui': ('VERB', 'ser', 'fui', {'Person': '1', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Ind'}),
        'foste': ('VERB', 'ser', 'foste', {'Person': '2', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Ind'}),
        'foi': ('VERB', 'ser', 'foi', {'Person': '3', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Ind'}),
        'fomos': ('VERB', 'ser', 'fomos', {'Person': '1', 'Number': 'Plur', 'Tense': 'Past', 'Mood': 'Ind'}),
        'fostes': ('VERB', 'ser', 'fostes', {'Person': '2', 'Number': 'Plur', 'Tense': 'Past', 'Mood': 'Ind'}),
        'foram': ('VERB', 'ser', 'foram', {'Person': '3', 'Number': 'Plur', 'Tense': 'Past', 'Mood': 'Ind'}),
        'serei': ('VERB', 'ser', 'serei', {'Person': '1', 'Number': 'Sing', 'Tense': 'Fut', 'Mood': 'Ind'}),
        'serás': ('VERB', 'ser', 'serás', {'Person': '2', 'Number': 'Sing', 'Tense': 'Fut', 'Mood': 'Ind'}),
        'será': ('VERB', 'ser', 'será', {'Person': '3', 'Number': 'Sing', 'Tense': 'Fut', 'Mood': 'Ind'}),
        'seremos': ('VERB', 'ser', 'seremos', {'Person': '1', 'Number': 'Plur', 'Tense': 'Fut', 'Mood': 'Ind'}),
        'sereis': ('VERB', 'ser', 'sereis', {'Person': '2', 'Number': 'Plur', 'Tense': 'Fut', 'Mood': 'Ind'}),
        'serão': ('VERB', 'ser', 'serão', {'Person': '3', 'Number': 'Plur', 'Tense': 'Fut', 'Mood': 'Ind'}),
        'seja': ('VERB', 'ser', 'seja', {'Person': '1', 'Number': 'Sing', 'Tense': 'Pres', 'Mood': 'Sub'}),
        'sejas': ('VERB', 'ser', 'sejas', {'Person': '2', 'Number': 'Sing', 'Tense': 'Pres', 'Mood': 'Sub'}),
        'seja': ('VERB', 'ser', 'seja', {'Person': '3', 'Number': 'Sing', 'Tense': 'Pres', 'Mood': 'Sub'}),
        'sejamos': ('VERB', 'ser', 'sejamos', {'Person': '1', 'Number': 'Plur', 'Tense': 'Pres', 'Mood': 'Sub'}),
        'sejais': ('VERB', 'ser', 'sejais', {'Person': '2', 'Number': 'Plur', 'Tense': 'Pres', 'Mood': 'Sub'}),
        'sejam': ('VERB', 'ser', 'sejam', {'Person': '3', 'Number': 'Plur', 'Tense': 'Pres', 'Mood': 'Sub'}),
        'fosse': ('VERB', 'ser', 'fosse', {'Person': '1', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Sub'}),
        'fosses': ('VERB', 'ser', 'fosses', {'Person': '2', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Sub'}),
        'fosse': ('VERB', 'ser', 'fosse', {'Person': '3', 'Number': 'Sing', 'Tense': 'Past', 'Mood': 'Sub'}),
        'fôssemos': ('VERB', 'ser', 'fôssemos', {'Person': '1', 'Number': 'Plur', 'Tense': 'Past', 'Mood': 'Sub'}),
        'fossem': ('VERB', 'ser', 'fossem', {'Person': '3', 'Number': 'Plur', 'Tense': 'Past', 'Mood': 'Sub'}),
        'for': ('VERB', 'ser', 'for', {'Person': '1', 'Number': 'Sing', 'Tense': 'Fut', 'Mood': 'Sub'}),
        'fores': ('VERB', 'ser', 'fores', {'Person': '2', 'Number': 'Sing', 'Tense': 'Fut', 'Mood': 'Sub'}),
        'for': ('VERB', 'ser', 'for', {'Person': '3', 'Number': 'Sing', 'Tense': 'Fut', 'Mood': 'Sub'}),
        'formos': ('VERB', 'ser', 'formos', {'Person': '1', 'Number': 'Plur', 'Tense': 'Fut', 'Mood': 'Sub'}),
        'forem': ('VERB', 'ser', 'forem', {'Person': '3', 'Number': 'Plur', 'Tense': 'Fut', 'Mood': 'Sub'})
    }
    if token.lower() in verb_flexions:
        return verb_flexions[token.lower()]

    if token.lower().endswith(('ar', 'er', 'ir', 'ado', 'ido', 'ando', 'endo', 'indo', 'ada')):
        return ('VERB', token, token, {})

    if token.lower().endswith(('vel', 'al', 'ar', 'oso', 'osa', 'ível')):
        return ('ADJ', token, token, {})

    aux_verbs = {'ser', 'estar', 'ter', 'haver'}
    if token.lower() in aux_verbs:
        return ('AUX', token, token, {})

    interjections = {'oh', 'ah', 'eh', 'ai', 'epa', 'olá', 'oi'}
    if token.lower() in interjections:
        return ('INTJ', token, token, {})

    particles = {'não', 'sim', 'talvez', 'quase', 'já', 'mesmo', 'ainda', 'até', 'bem', 'só', 'também', 'apenas'}
    if token.lower() in particles:
        return ('PART', token, token, {})
    else:
        return None
def preprocess_data(texts, pos_tags, lemmas, feats):
    preprocessed_texts = []
    preprocessed_pos_tags = []
    preprocessed_lemmas = []
    preprocessed_feats = []
    true_pos_tags = []
    true_lemmas = []
    true_feats = []
    for text, pos_tag, lemma, feat in zip(texts, pos_tags, lemmas, feats):
        result = preprocess_token(text)
        if result:
            preprocessed_texts.append(result[1])
            preprocessed_pos_tags.append(result[0])
            preprocessed_lemmas.append(result[2])
            preprocessed_feats.append(result[3])
            true_pos_tags.append(pos_tag)
            true_lemmas.append(lemma)
            true_feats.append(feat)
    return preprocessed_texts, preprocessed_pos_tags, preprocessed_lemmas, preprocessed_feats, true_pos_tags, true_lemmas, true_feats

def unprocessed_data(texts, pos_tags, lemmas, feats):
    unprocessed_texts = []
    unprocessed_pos_tags = []
    unprocessed_lemmas = []
    unprocessed_feats = []
    for text, pos_tag, lemma, feat in zip(texts, pos_tags, lemmas, feats):
        if not preprocess_token(text):
            unprocessed_texts.append(text)
            unprocessed_pos_tags.append(pos_tag)
            unprocessed_lemmas.append(lemma)
            unprocessed_feats.append(feat)
    return unprocessed_texts, unprocessed_pos_tags, unprocessed_lemmas, unprocessed_feats


1.2 Carregando e tratando dados de treinamento

1.2.1 Dataset de treino

In [10]:
# Carregar o corpus Bosque
bosque = CoNLLU(files=["pt_porttinari-ud-train.conllu"])

#separar features:y_pred
texts, pos_tags, lemmas, feats = extract_data(bosque)

df = pd.DataFrame({'text': texts, 'pos': pos_tags, 'lemma': lemmas, 'feats': feats})

dados_preprocessados_raw = preprocess_data(texts, pos_tags, lemmas, feats)
dados_nao_preprocessados = unprocessed_data(texts, pos_tags, lemmas, feats)

dados_preprocessados = pd.DataFrame(
    {'text': dados_preprocessados_raw[0],
     'pos': dados_preprocessados_raw[1],
     'lemma': dados_preprocessados_raw[2],
     'feats': dados_preprocessados_raw[3]
    }
)
valores_reais = pd.DataFrame(
    {'text': dados_preprocessados_raw[0],
     'pos': dados_preprocessados_raw[4],
     'lemma': dados_preprocessados_raw[5],
     'feats': dados_preprocessados_raw[6]
    }
)
dados_nao_preprocessados = pd.DataFrame({
    'text': dados_nao_preprocessados[0],
    'pos': dados_nao_preprocessados[1],
    'lemma': dados_nao_preprocessados[2],
    'feats': dados_nao_preprocessados[3]
    }
)
print(dados_preprocessados.shape) # quantidade de tokens que foram pré-processados
dados_nao_preprocessados = dados_nao_preprocessados[dados_nao_preprocessados['pos'] != 'X']
dados_nao_preprocessados = dados_nao_preprocessados[dados_nao_preprocessados['pos'] != 'PUNCT']


(82829, 4)


1.2.2 Dataset de teste

In [11]:
test = CoNLLU(files=["pt_porttinari-ud-test.conllu"])
texts_test, pos_tags_test, lemmas_test, feats_test = extract_data(test)
df_test = pd.DataFrame({'text': texts_test, 'pos': pos_tags_test, 'lemma': lemmas_test, 'feats': feats_test})
tamanhoTotal = len(df_test)
dados_preprocessados_test_raw = preprocess_data(texts_test, pos_tags_test, lemmas_test, feats_test)
dados_nao_preprocessados_test = unprocessed_data(texts_test, pos_tags_test, lemmas_test, feats_test)
dados_preprocessados_test = pd.DataFrame(
    {'text': dados_preprocessados_test_raw[0],
     'pos': dados_preprocessados_test_raw[1],
     'lemma': dados_preprocessados_test_raw[2],
     'feats': dados_preprocessados_test_raw[3]
    }
)
tamanhoPreProcessados = len(dados_preprocessados_test)
valores_reais_test = pd.DataFrame(
    {'text': dados_preprocessados_test_raw[0],
     'pos': dados_preprocessados_test_raw[4],
     'lemma': dados_preprocessados_test_raw[5],
     'feats': dados_preprocessados_test_raw[6]
    })
dados_nao_preprocessados_test = pd.DataFrame({
    'text': dados_nao_preprocessados_test[0],
    'pos': dados_nao_preprocessados_test[1],
    'lemma': dados_nao_preprocessados_test[2],
    'feats': dados_nao_preprocessados_test[3]
})
tamanhoNaoPreProcessados = len(dados_nao_preprocessados_test)
dados_nao_preprocessados_test = dados_nao_preprocessados_test[dados_nao_preprocessados_test['pos'] != 'X']
dados_nao_preprocessados_test = dados_nao_preprocessados_test[dados_nao_preprocessados_test['pos'] != 'PUNCT']

In [12]:
print(dados_preprocessados.shape)
print(dados_nao_preprocessados.shape)
print(dados_preprocessados_test.shape)
print(dados_nao_preprocessados_test.shape)

(82829, 4)
(34998, 4)
(23709, 4)
(9817, 4)


1.3 Acurácia do Pré-Processamento

In [13]:
def calcular_acuracia(valores_reais,df_preprocessado,target):
    acertos=0
    erros=0
    if target=='pos':
        y_true=valores_reais['pos']
        y_pred=df_preprocessado['pos']
        for i in range(len(y_true)):
            if y_true[i]!=y_pred[i]:
                erros+=1
            else:
                acertos+=1
    elif target=='lemma':
        y_true=valores_reais['lemma']
        y_pred=df_preprocessado['lemma']
        for i in range(len(y_true)):
            if y_true[i]!=y_pred[i]:
                erros+=1
            else:
                acertos+=1
    elif target=='feats':
        y_true=valores_reais['feats']
        y_pred=df_preprocessado['feats']
        for i in range(len(y_true)):
            if y_true[i]!=y_pred[i]:
                erros+=1
            else:
                acertos+=1
    return acertos/(acertos+erros)
acuracia_pos_pre=calcular_acuracia(valores_reais_test,dados_preprocessados_test,'pos')
acuracia_lemma_pre=calcular_acuracia(valores_reais_test,dados_preprocessados_test,'lemma')
acuracia_feats_pre=calcular_acuracia(valores_reais_test,dados_preprocessados_test,'feats')
print(acuracia_pos_pre)
print(acuracia_lemma_pre)
print(acuracia_feats_pre) ## acurácia do modelo pré-processado pra feats é muito baixa,portanto aplicar aprendizado de máquina somente para este caso

0.784596566704627
0.797460879834662
0.5748028174954659


##Machine Learning

#POS-TAGGING:Análise de classes

Treinamento do modelo usando os dados não pré-processados

In [ ]:
#Separando dados para treinamento do POS-TAGGING
X_train = dados_nao_preprocessados['text']
y_train = dados_nao_preprocessados['pos']
X_test = dados_nao_preprocessados_test['text']
y_test = dados_nao_preprocessados_test['pos']
# Definir o pipeline
pipeline_pos = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', DecisionTreeClassifier(criterion='gini', min_samples_split=10))
])
# Treinar o modelo
pipeline_pos.fit(X_train, y_train)
# Fazer predições
y_pred = pipeline_pos.predict(X_test)
#salvando predicoes num dataframe
unprocess_pred = pd.DataFrame({'text': X_test, 'pos': y_pred})

In [ ]:
print(unprocess_pred.head())

          text   pos
0         você  PRON
1      recebeu  VERB
2      notícia  NOUN
3        seria   AUX
4  substituído  VERB


Avaliando modelo

In [ ]:
# Avaliar o modelo no conjunto de treinamento
train_accuracy = pipeline_pos.score(X_train, y_train)
#Medindo a acurácia no conjunto de teste
test_accuracy = pipeline_pos.score(X_test, y_test)
print('Acurácia no conjunto de treinamento:', train_accuracy)
print('Acurácia no conjunto de teste:', test_accuracy)
acuracia_pos=test_accuracy

Acurácia no conjunto de treinamento: 0.9463186356073211
Acurácia no conjunto de teste: 0.7837383660806618


#LEMMATIZACAO

In [ ]:
x_train = pd.Series(dados_nao_preprocessados['text'])
y_train = pd.Series(dados_nao_preprocessados['lemma'])
x_test = pd.Series(dados_nao_preprocessados_test['text'])
y_test = pd.Series(dados_nao_preprocessados_test['lemma'])
print(X_train.tail())
print(y_train.tail())
print(X_train.shape)
print(y_train.shape)

48162    concentrados
48163      campeonato
48164            onde
48165          ocupam
48166        invictos
Name: text, dtype: object
48162    concentrado
48163     campeonato
48164           onde
48165         ocupar
48166        invicto
Name: lemma, dtype: object
(48080,)
(48080,)


In [ ]:

# Definir o pipeline
pipeline_lemma = Pipeline([
    ('vectorizer', CountVectorizer()),
    ('classifier', DecisionTreeClassifier(criterion='gini', min_samples_split=10))
])
# Treinar o modelo
pipeline_lemma.fit(X_train, y_train)
# Fazer predições
y_pred = pipeline_lemma.predict(X_test)
#salvando predicoes num dataframe
unprocess_pred = unprocess_pred.join(pd.DataFrame({'lemma': y_pred}))

In [ ]:
train_accuracy = pipeline_lemma.score(X_train, y_train)
print('Acurácia no conjunto de treinamento:', train_accuracy)
#Medindo a acurácia no conjunto de teste
test_accuracy = pipeline_lemma.score(X_test, y_test)
print('Acurácia no conjunto de teste:', test_accuracy)
acuracia_lemma=test_accuracy

Acurácia no conjunto de treinamento: 0.972608153078203
Acurácia no conjunto de teste: 0.8085573940020683


#Analisador de flexão

Normalizando feats e analisando dados


In [ ]:
X_train = df['text']
y_train_raw = df['feats']
X_test = df_test['text']
y_test_raw = df_test['feats']

In [ ]:
print(X_train.head())

0          PT
1          em
2           o
3     governo
4    BRASÍLIA
Name: text, dtype: object


In [ ]:
y_train_raw = pd.Series(y_train_raw)
y_test_raw = pd.Series(y_test_raw)
print(X_train.shape)
print(y_train_raw.shape)


(171776,)
(171776,)


In [ ]:
print(y_train_raw.head())

0                 {'Gender': 'Masc', 'Number': 'Sing'}
1                                                   {}
2    {'Definite': 'Def', 'Gender': 'Masc', 'Number'...
3                 {'Gender': 'Masc', 'Number': 'Sing'}
4                  {'Gender': 'Fem', 'Number': 'Sing'}
Name: feats, dtype: object


In [ ]:
y_train = pd.json_normalize(y_train_raw)
y_test = pd.json_normalize(y_test_raw)
y_train = y_train.fillna('None')
y_test = y_test.fillna('None')
common_columns = set(y_train.columns).intersection(set(y_test.columns))
y_train = y_train[sorted(common_columns)]
y_test = y_test[sorted(common_columns)]
print(y_train.head())

   Abbr  Case Definite ExtPos Gender  Mood NumType Number Person Polarity  \
0  None  None     None   None   Masc  None    None   Sing   None     None   
1  None  None     None   None   None  None    None   None   None     None   
2  None  None      Def   None   Masc  None    None   Sing   None     None   
3  None  None     None   None   Masc  None    None   Sing   None     None   
4  None  None     None   None    Fem  None    None   Sing   None     None   

  PronType Tense  Typo VerbForm Voice  
0     None  None  None     None  None  
1     None  None  None     None  None  
2      Art  None  None     None  None  
3     None  None  None     None  None  
4     None  None  None     None  None  


In [ ]:
#Selecionar colunas desejadas
y_train = y_train[['Gender', 'Number', 'Person', 'Tense', 'VerbForm', 'PronType']]
y_test = y_test[['Gender', 'Number', 'Person', 'Tense', 'VerbForm', 'PronType']]

Conclusões da análise do dataset:
* target possuem muitas colunas de dados do tipo categórico que seguem regras,logo seria interessente dividir o treinamento em c colunas,aplicar one hot encoder e treinar usando árvores para cada coluna e ao fim juntar todas as colunas previstas.

In [ ]:

#transformando as features em vetores
#onde armazenaremos a acurácia de cada modelo e os modelos treinados
y_train = np.array(y_train)
y_test = np.array(y_test)
modelos = []
ys_pred_test = []
encoders = []
#aplicando one hot encoder(dados categoricos) e treinando os modelos
for col in range(y_train.shape[1]):
    #aplicando one hot encoder
    enc = OneHotEncoder(handle_unknown='ignore')
    y_train_enc = enc.fit_transform(y_train[:, col].reshape(-1, 1)).toarray()
    y_test_enc = enc.transform(y_test[:, col].reshape(-1, 1)).toarray()
    #treinando o modelo
    pipeline_feats = Pipeline([
        ('vectorizer', CountVectorizer()),
        ('classifier', DecisionTreeClassifier(criterion='gini', min_samples_split=10))
    ])
    pipeline_feats.fit(X_train, y_train_enc)
    modelos.append(pipeline_feats)
    #medindo a acurácia
    y_pred_train = pipeline_feats.predict(X_train)
    train_accuracy = pipeline_feats.score(X_train, y_train_enc)
    print('Acurácia no conjunto de treinamento:', train_accuracy)
    y_pred_test = pipeline_feats.predict(X_test)
    test_accuracy = pipeline_feats.score(X_test, y_test_enc)
    print('Acurácia no conjunto de teste:', test_accuracy)
    #convertendo as previões para o formato original
    y_pred_test = enc.inverse_transform(y_pred_test)
    ys_pred_test.append(y_pred_test)
    encoders.append(enc)


Acurácia no conjunto de treinamento: 0.8309542660208644
Acurácia no conjunto de teste: 0.7729676858426315
Acurácia no conjunto de treinamento: 0.8539260432190761
Acurácia no conjunto de teste: 0.7769526155629619
Acurácia no conjunto de treinamento: 0.986395072652757
Acurácia no conjunto de teste: 0.9739168236487465
Acurácia no conjunto de treinamento: 0.9902023565573771
Acurácia no conjunto de teste: 0.9794594986233879
Acurácia no conjunto de treinamento: 0.9884093237704918
Acurácia no conjunto de teste: 0.9650775249963773
Acurácia no conjunto de treinamento: 0.8853914400149031
Acurácia no conjunto de teste: 0.8857411969279815


Calculo da acurácia do modelo

In [ ]:
#Realizando a predição no conjunto de teste
for i in range(len(ys_pred_test)):
    ys_pred_test[i] = pd.DataFrame(ys_pred_test[i], columns=[f'feats_{i}'])
y_pred_test = pd.concat(ys_pred_test, axis=1)
y_pred_test = y_pred_test.fillna('None')
print(y_pred_test.head())
y_test = pd.DataFrame(y_test, columns=[f'feats_{i}' for i in range(y_test.shape[1])])
print(y_test.head())
#Calculando a acurácia
acertos, erros = 0, 0
for i in range(y_test.shape[0]):
    if y_test.iloc[i].equals(y_pred_test.iloc[i]):
        acertos += 1
    else:
        erros += 1
accuracy = acertos/(acertos+erros)
print(f'Acurácia no conjunto de teste: {accuracy:.2f}')
acuracia_feats=accuracy


  feats_0 feats_1 feats_2 feats_3 feats_4 feats_5
0     Fem    Sing    None    None    None    None
1    None    None    None    None    None    None
2    None    None    None    None    None    None
3    None    Sing       3    None    None     Prs
4    None    Sing       3    Past     Fin    None
  feats_0 feats_1 feats_2 feats_3 feats_4 feats_5
0     Fem    Sing    None    None    None    None
1    None    None    None    None    None    None
2    None    None    None    None    None    None
3    None    Sing       3    None    None     Prs
4    None    Sing       3    Past     Fin    None
Acurácia no conjunto de teste: 0.73


Organizando os datasets de teste e previsão para calcular a acurácia


In [ ]:
y_pred_test.columns = ['Gender', 'Number', 'Person', 'Tense', 'VerbForm', 'PronType']
y_test.columns = ['Gender', 'Number', 'Person', 'Tense', 'VerbForm', 'PronType']
print(y_pred_test.head())
print(y_test.head())

  Gender Number Person Tense VerbForm PronType
0    Fem   Sing   None  None     None     None
1   None   None   None  None     None     None
2   None   None   None  None     None     None
3   None   Sing      3  None     None      Prs
4   None   Sing      3  Past      Fin     None
  Gender Number Person Tense VerbForm PronType
0    Fem   Sing   None  None     None     None
1   None   None   None  None     None     None
2   None   None   None  None     None     None
3   None   Sing      3  None     None      Prs
4   None   Sing      3  Past      Fin     None


In [ ]:
df_test = df_test.join(y_test)
df_test = df_test.drop(columns=['feats'])
df_test = df_test.fillna('None')

In [ ]:
print(df_test.head())

      text    pos    lemma Gender Number Person Tense VerbForm PronType
0    Folha  PROPN    Folha    Fem   Sing   None  None     None     None
1       --  PUNCT       --   None   None   None  None     None     None
2     Como    ADV     como   None   None   None  None     None     None
3     você   PRON     você   None   Sing      3  None     None      Prs
4  recebeu   VERB  receber   None   Sing      3  Past      Fin     None


In [ ]:
previsoes = pd.DataFrame({'text': X_test, 'pos': None, 'lemma': None})
previsoes = previsoes.join(y_pred_test)
print(previsoes.head())

      text   pos lemma Gender Number Person Tense VerbForm PronType
0    Folha  None  None    Fem   Sing   None  None     None     None
1       --  None  None   None   None   None  None     None     None
2     Como  None  None   None   None   None  None     None     None
3     você  None  None   None   Sing      3  None     None      Prs
4  recebeu  None  None   None   Sing      3  Past      Fin     None


In [ ]:
for i in range(df_test.shape[1]):
    if dados_nao_preprocessados_test['text'][i]==previsoes['text'][i]:
        previsoes['pos'][i] = unprocess_pred['pos'][i]
        previsoes['lemma'][i] = unprocess_pred['lemma'][i]
    else:
        previsoes['pos'][i] = dados_preprocessados_test['pos'][i]
        previsoes['lemma'][i] = dados_preprocessados_test['lemma'][i]
print(previsoes.head())
print(df_test.head())

      text    pos  lemma Gender Number Person Tense VerbForm PronType
0    Folha  PROPN  Folha    Fem   Sing   None  None     None     None
1       --    SYM     --   None   None   None  None     None     None
2     Como  SCONJ   Como   None   None   None  None     None     None
3     você    ADP      a   None   Sing      3  None     None      Prs
4  recebeu    ADP     de   None   Sing      3  Past      Fin     None
      text    pos    lemma Gender Number Person Tense VerbForm PronType
0    Folha  PROPN    Folha    Fem   Sing   None  None     None     None
1       --  PUNCT       --   None   None   None  None     None     None
2     Como    ADV     como   None   None   None  None     None     None
3     você   PRON     você   None   Sing      3  None     None      Prs
4  recebeu   VERB  receber   None   Sing      3  Past      Fin     None


In [ ]:
print(previsoes.shape)
print(df_test.shape)

(27604, 9)
(27604, 9)


In [ ]:
#feats que serão consideradas:Gender,Number,Person,Tense,VerbForm,PronType
# Seleção das colunas específicas
df_test_selected = df_test[['text', 'pos', 'lemma', 'Gender', 'Number', 'Person', 'Tense', 'VerbForm', 'PronType']]
previsoes_selected = previsoes[['text', 'pos', 'lemma', 'Gender', 'Number', 'Person', 'Tense', 'VerbForm', 'PronType']]

# Impressão das primeiras linhas dos DataFrames selecionados
print(df_test_selected.head())
print(previsoes_selected.head())


      text    pos    lemma Gender Number Person Tense VerbForm PronType
0    Folha  PROPN    Folha    Fem   Sing   None  None     None     None
1       --  PUNCT       --   None   None   None  None     None     None
2     Como    ADV     como   None   None   None  None     None     None
3     você   PRON     você   None   Sing      3  None     None      Prs
4  recebeu   VERB  receber   None   Sing      3  Past      Fin     None
      text    pos  lemma Gender Number Person Tense VerbForm PronType
0    Folha  PROPN  Folha    Fem   Sing   None  None     None     None
1       --    SYM     --   None   None   None  None     None     None
2     Como  SCONJ   Como   None   None   None  None     None     None
3     você    ADP      a   None   Sing      3  None     None      Prs
4  recebeu    ADP     de   None   Sing      3  Past      Fin     None


In [ ]:
previsoes_ord = previsoes.sort_values(by='text')
previsoes_ord = previsoes_ord.reset_index(drop=True)
previsoes_ord = previsoes_ord[df_test.columns]

#Calculando a acurácia total

In [ ]:
def calcular_acuracia_total(df_test, previsoes):
    acertos = 0
    total_atributos = 8  # Total de atributos a serem comparados

    for i in range(len(df_test)):
        acertos_item = 0

        if df_test['pos'][i] == previsoes['pos'][i]:
            acertos_item += 1

        if df_test['lemma'][i] == previsoes['lemma'][i]:
            acertos_item += 1

        if df_test['Gender'][i] == previsoes['Gender'][i]:
            acertos_item += 1

        if df_test['Number'][i] == previsoes['Number'][i]:
            acertos_item += 1

        if df_test['Person'][i] == previsoes['Person'][i]:
            acertos_item += 1

        if df_test['Tense'][i] == previsoes['Tense'][i]:
            acertos_item += 1

        if df_test['VerbForm'][i] == previsoes['VerbForm'][i]:
            acertos_item += 1

        if df_test['PronType'][i] == previsoes['PronType'][i]:
            acertos_item += 1

        acertos += acertos_item

    total_comparacoes = len(df_test) * total_atributos
    acuracia = acertos / total_comparacoes
    return acuracia
print(f'A acurácia obtida foi de {calcular_acuracia_total(df_test_selected, previsoes_selected)*100}%')

A acurácia obtida foi de 67.26968917548182%


#Criando o Analisador Morfológico

In [ ]:
import pandas as pd

def AnalisadorMorfológico(texto):
    lemmas = []
    pos_tags = []
    feats = []

    feats_template = {
        'Gender': None,
        'Number': None,
        'Person': None,
        'Tense': None,
        'VerbForm': None,
        'PronType': None
    }

    # Lista das chaves de características na ordem dos modelos
    chaves_caracteristicas = list(feats_template.keys())

    # Tokenizando a string de entrada
    if isinstance(texto, str):
        texto = texto.split()

    for token in texto:
        result = preprocess_token(token)
        if result:
            pos_tags.append(result[0])
            lemmas.append(result[2])
        else:
            lemmas.append(pipeline_lemma.predict([token])[0])
            pos_tags.append(pipeline_pos.predict([token])[0])

        token_feats = {}
        for i, model in enumerate(modelos):
            chave = chaves_caracteristicas[i]
            predicao_onehot = model.predict([token])
            predicao_label = encoders[i].inverse_transform(predicao_onehot)[0]
            if predicao_label is not None:
                token_feats[chave] = predicao_label

        feats.append(token_feats)

    # Criando um DataFrame para facilitar a visualização
    output_df = pd.DataFrame({
        'Token': texto,
        'Lemma': lemmas,
        'POS': pos_tags,
    })

    feats_df = pd.DataFrame(feats)
    output_df = pd.concat([output_df, feats_df], axis=1)

    return output_df

# Exemplo de uso
analisador = AnalisadorMorfológico('estou estudando processamento de linguagem natural')
print(analisador)


           Token          Lemma   POS  Gender  Number  Person   Tense  \
0          estou          estar   AUX  [None]  [Sing]     [1]  [Pres]   
1      estudando      estudando  VERB  [None]  [None]  [None]  [None]   
2  processamento  processamento  NOUN  [Masc]  [Sing]  [None]  [None]   
3             de             de   ADP  [None]  [None]  [None]  [None]   
4      linguagem      linguagem  NOUN   [Fem]  [Sing]  [None]  [None]   
5        natural        natural   ADJ  [None]  [Sing]  [None]  [None]   

  VerbForm PronType  
0    [Fin]   [None]  
1   [None]   [None]  
2   [None]   [None]  
3   [None]   [None]  
4   [None]   [None]  
5   [None]   [None]  
